In [1]:
import pandas as pd
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter  # 专门提供经纬度的
from cartopy.util import add_cyclic_point
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib.patheffects as path_effects
import matplotlib.path as mpath
from cnmaps import get_adm_maps, draw_maps
from matplotlib import ticker
import cmaps
from matplotlib.ticker import MultipleLocator, FixedLocator
from eofs.standard import Eof
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.ndimage import filters
from tqdm import tqdm
import geopandas as gpd
import salem
from toolbar.TN_WaveActivityFlux import TN_WAF_3D, TN_WAF
from toolbar.curved_quivers.modplot import Curlyquiver
import pprint


C:\Users\86136\.conda\envs\Meteorological\lib\site-packages\cnmaps\__init__.py:23: UserWarning: 由于Shapely的版本高于2.0.0, 请将Shapely的版本降级到2.0.0以下, 否则会影响功能的使用
  warnings.warn(("由于Shapely的版本高于2.0.0, 请将Shapely的版本降级到2.0.0以下, 否则会影响功能的使用"))


In [ ]:
reg_z200 = [[np.polyfit(sat_78[:], z200_78[:, ilat, ilon]/9.8,1)[0] for ilon in range(len(lon_uvz))] for ilat in tqdm(range(len(lat_uvz)), desc='计算LBM 2mT Z200', position=0, leave=True)]